# aapf_map_viz.ipynb
Let's visualize where the NSF Astronomy & Astrophysics Postdoctoral Fellowship recipients took their $$$

In [1]:
import pandas as pd
import numpy as np

NSF data available at [this really long link](https://www.nsf.gov/awardsearch/advancedSearchResult?PIId=&PIFirstName=&PILastName=&PIOrganization=&PIState=&PIZip=&PICountry=&ProgOrganization=&ProgEleCode=1609&BooleanElement=All&ProgRefCode=&BooleanRef=All&Program=&ProgOfficer=&Keyword=&AwardNumberOperator=&AwardAmount=&AwardInstrument=&ActiveAwards=true&ExpiredAwards=true&OriginalAwardDateOperator=&StartDateOperator=&ExpDateOperator=). Thanks NSF !

In [2]:
## we don't trust pd.read_csv in this household
with open("data/Awards.csv",'rb') as handle:
    data = [line[1:-1].split('","') for line in 
            handle.read().decode(errors='ignore').split('\n')[:-1]]
    header = data[0]
    data_dict = dict(zip(header,np.array(data[1:]).T))
    aapf_df = pd.DataFrame(data_dict)

US city latitude and longitude data downloaded for free from: https://simplemaps.com/data/us-cities.

Thanks https://simplemaps.com !

In [3]:
## we don't trust pd.read_csv in this household
with open("data/uscities.csv",'r') as handle:
    data = [line[1:-1].split('","') for line in 
            handle.read().split('\n')[:-1]]
    header = data[0]
    data_dict = dict(zip(header,np.array(data[1:]).T))
    cities_df = pd.DataFrame(data_dict)

## take advantage of the fact this dataset has state abbrev. and state names
##  to make a mapping dictionary for later
mappings = np.unique(list(zip(list(cities_df['state_id']),list(cities_df['state_name']))),axis=0)
state_mapper = dict(mappings)

Zipcode latitude and longitude data downloaded from the [US Census Bureau](https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html). 

Thanks US Census Bureau!

In [4]:
## we don't trust pd.read_csv in this household
with open("data/2021_Gaz_zcta_national.txt",'r') as handle:
    data = [line.split('\t') for line in 
            handle.read().split('\n')[:-1]]
    header = data[0]
    data_dict = dict(zip(header,np.array(data[1:]).T))
    zip_df = pd.DataFrame(data_dict)

## reduce the dataset just to lat and longitude and set the index
##  to the zipcode so that we can use the dataframe as a lookup table
zip_df.set_index('GEOID',inplace=True)
zip_df = zip_df.T[-2:].T

# Alright, time to parse the data!

In [5]:
lat_lons = []
for index,postdoc in aapf_df.iterrows():
    ## some organization zips are the long boys with extra digits
    zipcode = postdoc['OrganizationZip'][:5]
    
    ## ah yes, the sweet sweet ideal scenario, if only it were always so
    if zipcode in zip_df.index: 
        lat_lons += [tuple(zip_df.loc[zipcode].values)]
    
    ## okay let's try looking by city instead since we can't match by zip code
    else:
        city = postdoc['OrganizationCity'].title()
        
        ## oh NSF, you put Toronto, Ontario into the city field...
        if 'Toronto' in city: city = city.split(',')[0]
        
        matches = cities_df[cities_df['city_ascii'] == city]
        
        ## cry, there aren't any city matches
        if matches.shape[0] == 0:
            lat_lons += [(np.nan,np.nan)]
            print('$$ gone to:', postdoc['OrganizationCity'],postdoc['OrganizationState'])

        ## exactly one match, hurray!
        elif matches.shape[0] == 1: lat_lons += [(matches['lat'].values[0],matches['lng'].values[0])]
        ## multiple cities in the US with this name, need to match on state
        else: 
            sub_matches = matches[postdoc['OrganizationState'] == matches['state_id']]
            ## oh Exeter & Toronto, why are there US cities named after you?
            if sub_matches.shape[0] == 0:
                print('$$ gone to:', postdoc['OrganizationCity'],postdoc['OrganizationState'],'(but almost had you w/ a matching US city!)')
                lat_lons += [(np.nan,np.nan)]
            ## one match! we did it
            elif sub_matches.shape[0] == 1: 
                lat_lons += [(sub_matches['lat'].values[0],sub_matches['lng'].values[0])]
            ## multiple cities in this state with the same name? is that... possible?
            else:
                print(postdoc['OrganizationCity'],postdoc['OrganizationState'])
                print(matches.shape,matches[['city','state_id','state_name']])
                lat_lons += [(np.nan,np.nan)]

## convert to numpy array
lat_lons = np.array(lat_lons,dtype=np.float64)

$$ gone to: Garching 
$$ gone to: Sydney 
$$ gone to: Exeter  (but almost had you w/ a matching US city!)
$$ gone to: 1091 WS Amsterdam 
$$ gone to: Kaleden, BC 
$$ gone to:  
$$ gone to: CH-8049 Zurich 
$$ gone to: Toronto, Ontario, M5S3H8  (but almost had you w/ a matching US city!)


### Now let's extract the institution from the abstract
Thanks NSF for formatting your abstracts (relatively) uniformly!

In [6]:
locations = []
for abstract in aapf_df['Abstract']:
    ## early on their format was something like ___ was awarded a fellowship
    ##  to do ____ at ____ and like 50% of the time they put Dr. in the first ___
    my_abstract = abstract.split('awarded')[1] if 'awarded' in abstract else abstract

    ## split on periods to get a list of sentences
    abs_first_sent = my_abstract[:].split('.')

    ## sometimes, despite our best efforts, the first "sentence" is still
    ##  a title like Dr., Mr., Ms. , etc.
    abs_first_sent = abs_first_sent[(len(abs_first_sent[0]) < 15)]
    
    ## by some miracle they really put at ____ for *every* abstract
    ##  and what immediately follows is the institution, incredible.
    location = abs_first_sent.split(' at ')
    ## sometimes there's multiple institutions so we want to get them all
    location = ' at '.join(location[1:])
    ## sorry ohio state
    location = location.replace('the ','')

    ## do a little bit of data validation to make sure we actually got what we wanted
    ##  would need to update this list if a legit institution was raising an error
    flag = False
    for key in ['University','Museum','Center',
                'Planetarium','College','Institute',
                'universities','Institution','Caltech',
                'Universities','Observatory']: 
        flag = flag or key in location
    if not flag: raise KeyError(location,abs_first_sent)
    
    locations += [location]

## literally only the last one had additional content between the 'at ' and the '.' besides
##  the institution. At least it was separate by a comma.
locations[-1] = locations[-1].split(',')[0]

## Need to flag locations which are not geocoded correctly. See cells at bottom for how we did this manually

In [7]:
flags = [False, True, False, True, True, False, True, False, False, True, False, True, True, True, True, True, False, True, True, False, True, True, False, False, True, True, True, True, False, False, True, True, True, True, False, True, False, True, True, True, True, False, False, False, True, True, False, True, False, True, True, True, True, False, False, False, True, True, True, False, True, False, True, True, True, True, True, False, True, False, True, True, False, True, True, True, False, True, False, True, True, False, False, False, False, True, True, True, True, True, True, True, False, False, False, True, False, True, True, True, True, True, False, True, False, True, True, True, False, False, False, True, True, True, True, False, False, True, False, False, False, False, False, True, False, True, False, True, True, False, False, True, False, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, True, False, False, True, False, False, True, False, True, False, False, False, False, False, False, True, False, True, False, True, False, False, False, False, True, True, False, True, False, False, True, False, False, True, True, False, True, False]

In [8]:
## stuff that was messed up and I had to fix manually: 
flags[64] = False ## i said JHU wasn't in maryland above, whoops
flags[185] = True ## bad location for amherst, I guess they started in boston

In [9]:
states = [state_mapper[state] if state in state_mapper else '_NAN_' for state in aapf_df['OrganizationState'].tolist()]

## start with NMSU because there's a correctly geocoded entry for ___ and NMSU that we don't want to find first
## when we do the remapping... so this should be the first key to guarantee that won't happen
good_remapper = {'New Mexico State University':('New Mexico',(32.2793, -106.7491))}
good_remapper['University of California, Santa Barbara'] = ('California',(34.414253, -119.840902))
good_remapper.update({location:(state,lat_lon) for (location,flag,state,lat_lon) in zip(locations,flags,states,lat_lons) if not flag})


## manually have to add ~20 entries because these institutions only received 1
##  NSF AAPF or every instance of an NSF AAPF at this location has an incorrect city+state

## copy from ones that *did* work but won't match on their own
good_remapper['Institute for Advanced Study'] = good_remapper['Princeton University']
good_remapper['Caltech'] = good_remapper['California Institute of Technology']

## RIP Amherst
good_remapper['University of Massachusetts-Amherst'] = ('Massachusetts',(42.3868, -72.5301))
good_remapper['University of Massachusetts at Amherst'] = good_remapper['University of Massachusetts-Amherst']
good_remapper['University of Massachusetts, Amherst'] = good_remapper['University of Massachusetts-Amherst']

## multiple missing for IfA
good_remapper['University of Hawaii Institute for Astronomy (IfA)'] = ('Hawaii',(21.3072,-157.8116))
good_remapper['Institute for Astronomy at University of Hawaii'] = good_remapper['University of Hawaii Institute for Astronomy (IfA)']

## multiple missing for Santa Barbara

good_remapper['Institute for Theoretical Physics at University of California Santa Barbara (UCSB)'] = good_remapper['University of California, Santa Barbara']

good_remapper['Dartmouth College'] = ('New Hampshire', (43.7044, -72.2887))
good_remapper['University of Utah'] = ('Utah',(40.7649, -111.8421))
good_remapper['Carnegie Institution Department of Terrestrial Magnetism'] = ('Washington, D.C.',(38.958554, -77.063200))
good_remapper['Department of Terrestrial Magnetism of Carnegie Institution'] = ('Washington, D.C.',(38.958554, -77.063200))
good_remapper['Vanderbilt University'] = ('Tennessee',(36.1447, -86.8027))
good_remapper['Observatories of Carnegie Institution of Washington'] = ('California',(34.155698, -118.133363))
good_remapper['Pennsylvania State University'] = ('Pennsylvania',(40.7982, -77.8599))
good_remapper['Rutgers, State University of New Jersey'] = ('New Jersey',(40.5008, -74.4474))
good_remapper['National Center for Atmospheric Research (NCAR)'] = ('Colorado',(39.978203, -105.274773))
good_remapper['Cerro Tololo InterAmerican Observatory (CTIO) of National Optical Astronomy Observatories (NOAO), in La Serena, Chile'] = ('La Serena, Chile',(-30.169275, -70.806257))


## reflect CTIO across the equator to be in our USA centric map
good_remapper['Cerro Tololo InterAmerican Observatory (CTIO) of National Optical Astronomy Observatories (NOAO), in La Serena, Chile'] = ('La Serena, Chile',(30.169275, -70.806257))

In [10]:
new_states = []
new_lat_lons = []
for i,(location,flag,state,lat_lon) in enumerate(zip(locations,flags,states,lat_lons)):
    if flag: 
        found = False
        for key,value in good_remapper.items():
            if key in location or location in key:
                found = True
                #print(f"Matching {location} to {key}")
                #print(f"{state} -> {value[0]}")
                state = value[0]
                lat_lon = value[1]
                break
        if not found: print(i,location,state)
    
    ## add (remapped) state and lat_lon to our new list
    new_states += [state]
    new_lat_lons += [lat_lon]
        
        
print(len(lat_lons)-len(new_lat_lons),'unaccounted for')
new_lat_lons = np.array(new_lat_lons,dtype=float)

0 unaccounted for


In [11]:
text_boxes = []

for i,location in enumerate(locations):
    text_box = (
        f"{aapf_df['PrincipalInvestigator'].loc[i]}<br>{location}" +
        f"<br>{aapf_df['StartDate'].loc[i]} - {aapf_df['EndDate'].loc[i]}")
    
    text_boxes+=[text_box]
text_boxes = np.array(text_boxes)

expired = pd.Timestamp.today() > pd.DatetimeIndex(aapf_df['EndDate'])

In [13]:
nsf_fellows = np.array([
    np.array([year.split('/')[-1] for year in aapf_df['StartDate'].values]),
    np.repeat('NSF',aapf_df.shape[0]),
    aapf_df['PrincipalInvestigator'].values,
    locations,
    aapf_df['Abstract']
],dtype='str').T

np.save('nsf_fellows.npy',nsf_fellows)

## And now we can make our map!

In [14]:
import plotly.graph_objects as go

categories = {
    'expired': pd.Timestamp.today() > pd.DatetimeIndex(aapf_df['EndDate'])
    }

categories['active'] = ~categories['expired']

colors = ['red','blue']
fig = go.Figure(go.Scattergeo())
fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
for color,category,cat_mask in zip(colors,categories.keys(),categories.values()):
    marker_dict = {
        'color':color,
        'size':12,
        'opacity':0.25
    }
    fig.add_scattergeo(
        lat = new_lat_lons[cat_mask,0]+np.random.random(np.sum(cat_mask))*0.15,
        lon = new_lat_lons[cat_mask,1]+np.random.random(np.sum(cat_mask))*0.15,
        hoverinfo='text',
        text=text_boxes[cat_mask],
        marker=marker_dict,
        legendgroup=category,
        name=category)

fig.update_geos(
    scope='usa',
#    landcolor = 'blue',
    subunitcolor = 'black'
#    center=dict(lon=-90, lat=41),
#    projection_rotation=dict(lon=30, lat=30, roll=30),
#    lataxis_range=[-30,30], lonaxis_range=[-180, -90]
)

fig.update_layout(
    dragmode=False,
    legend={
        'title':'  <b>Status:</b>',
        'x':0.9,
        'y':0.85,
        'font':{'size':18}},
    title={
        'text':'<b>NSF AAPF Awardees</b>',
        'x':0.5,
        'y':0.95,
        'xref':'container',
        'yref':'container',
        'font':{'size':24}})
fig.add_annotation(
    text='toggle by clicking<br>legend entries',
    x=0.9,
    y=0.84,
    xref='paper',
    yref='paper',
    xanchor='left',
    yanchor='bottom',
    showarrow=False,)
fig.show()
fig.write_html("nsf_aapf.html")

## If you value your time, do not run the cells below. they are for safe keeping

-------

In [449]:
states = [state_mapper[state] if state in state_mapper else '_NAN_' for state in aapf_df['OrganizationState'].tolist()]
flags = []
for location,state in zip(locations,states):
    if state not in location:
        ## alright let's do it
        if state == '_NAN_': foo = 'n'
        else: foo = input(f"is: {location} in {state}?")
        flags += [foo == 'n']
    else:
        flags += [False]
print(np.sum(flags)/len(flags))

is: Michigan State University in Ohio? n
is: Northwestern University in Wisconsin? n
is: University of Chicago in Illinois? y
is: Harvard-Smithsonian Center for Astrophysics in Massachusetts? y
is: Columbia University in New York? y
is: University of Chicago in Arizona? n
is: Cornell University in New York? y
is: Dartmouth College in California? n
is: University of Arizona in Maryland? n
is: Johns Hopkins University in Wisconsin? n
is: University of Utah in New Mexico? n
is: Yale University in New Haven, Connecticut in Georgia? n
is: Northwestern University in Illinois? y
is: Northwestern University in Massachusetts? n
is: Yale University in California? n
is: Yale University in Connecticut? y
is: University of Texas in California? n
is: California Institute of Technology in Washington? n
is: University of California, Santa Cruz in Arizona? n
is: Institute for Advanced Study in Arizona? n
is: University of Wisconsin-Madison in Virginia? n
is: University of Colorado in Ohio? n
is: Univer

0.515625


In [450]:
print(flags)

[False, True, False, True, True, False, True, False, False, True, False, True, True, True, True, True, False, True, True, False, True, True, False, False, True, True, True, True, False, False, True, True, True, True, False, True, False, True, True, True, True, False, False, False, True, True, False, True, False, True, True, True, True, False, False, False, True, True, True, False, True, False, True, True, True, True, True, False, True, False, True, True, False, True, True, True, False, True, False, True, True, False, False, False, False, True, True, True, True, True, True, True, False, False, False, True, False, True, True, True, True, True, False, True, False, True, True, True, False, False, False, True, True, True, True, False, False, True, False, False, False, False, False, True, False, True, False, True, True, False, False, True, False, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, True, 